In [1]:
import requests
from arcpy import env
import json
import zipfile

In [12]:
# Establish folder path

folder_path = r'C:\\Users\\15612\\Documents\\GIS-5571\\Lab2\\'

In [2]:
# Establish a variable for the ArcGIS Pro project and the ArcGIS Pro map

project = arcpy.mp.ArcGISProject("CURRENT")
m = project.listMaps("Map")[0]

In [5]:
# Create a file geodatabase

arcpy.management.CreateFileGDB(
    folder_path,
    'Lab2_Part2'
)

<Result 'C:\\Users\\15612\\Documents\\GIS-5571\\Lab2\\Lab2_Part2.gdb'>

In [13]:
# Set the workspace to the created geodatabase and a spatial reference

GDB_path = folder_path + 'Lab2_Part2.gdb'
env.workspace = GDB_path

spatial_ref = arcpy.SpatialReference(4326)

In [14]:
# Create the point for Dora's house

dora = arcpy.PointGeometry(
    arcpy.Point(-92.148796, 44.127985),
    spatial_reference=spatial_ref
)

arcpy.management.CopyFeatures(
    dora,
    'Doras_Home'
)

<Result 'C:\\\\Users\\\\15612\\\\Documents\\\\GIS-5571\\\\Lab2\\\\Lab2_Part2.gdb\\Doras_Home'>

# Don't forget to remove your Google Places API Key!!!!

In [15]:
# Gather GeoJSON for North Picnic Area in Whitewater State Park

north_picnic = requests.get('https://maps.googleapis.com/maps/api/place/textsearch/json?query=Whitewater%20State%20Park%20North%20Picnic%20Area&inputtype=textquery&fields=fomatted_address%2Cgeometry&key=*API_KEY*')
Npicnic_json = north_picnic.json()
print(Npicnic_json)

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'formatted_address': '19206 MN-74, St Charles, MN 55972, United States', 'geometry': {'location': {'lat': 44.0543872, 'lng': -92.0448256}, 'viewport': {'northeast': {'lat': 44.05581302989273, 'lng': -92.04338092010728}, 'southwest': {'lat': 44.05311337010728, 'lng': -92.04608057989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/park-71.png', 'icon_background_color': '#4DB546', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/tree_pinlet', 'name': 'North Picnic area', 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112937281817958526101">Casen Raehtz</a>'], 'photo_reference': 'AcJnMuFGxijDYD0SMO5QuDKX6147p-QyvjLpzFQsIq2tAXi251nN3W0PqeZ4CkgtBd-tNfSft5-IqmtHS_rowb-x55LLGj11DijHIfVFFlXeA0r5pxL_186G-DaMbhuzKxzkdF1Hxap6MsDf7SecwInqtz4Il9tD6JzTLAHCMafFV6uRnrNs', 'width': 4032}], 'place_id': 'ChIJTeSfRg-X-YcRZM-pODlD2_

In [16]:
# Extract the latitude and longitudes from the GeoJSON

lat = Npicnic_json['results'][0]['geometry']['location']['lat']
lng = Npicnic_json['results'][0]['geometry']['location']['lng']

#print(lat)
#print(lng)

picnic_area = arcpy.PointGeometry(
    arcpy.Point(lng,lat),
    spatial_reference = spatial_ref
)

arcpy.management.CopyFeatures(
    picnic_area,
    'Picnic_Area'
)

<Result 'C:\\\\Users\\\\15612\\\\Documents\\\\GIS-5571\\\\Lab2\\\\Lab2_Part2.gdb\\Picnic_Area'>

In [63]:
# Gather land cover data from the Minnesota Geospatial Commons API

request = requests.get(
    'https://gisdata.mn.gov/api/3/action/package_show?id=biota-landcover-nlcd-mn-2019',
    verify = False
)
    
LC_gj = request.json()
print(LC_gj)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'help': 'https://gisdata.mn.gov/api/3/action/help_show?name=package_show', 'success': True, 'result': {'license_title': 'License not specified', 'maintainer': None, 'relationships_as_object': [], 'private': False, 'maintainer_email': None, 'num_tags': 1, 'id': 'df5e5f96-7347-4aae-8b97-8cf3cd352d68', 'metadata_created': '2022-02-16T09:03:08.539540', 'metadata_modified': '2022-09-01T08:09:22.951823', 'author': None, 'author_email': None, 'state': 'active', 'version': None, 'creator_user_id': '61044ca7-ee56-4019-aef7-f7c3b5e06e3a', 'type': 'dataset', 'resources': [{'mimetype': None, 'cache_url': None, 'hash': '', 'description': '', 'gdrsResGuid': '', 'cache_last_updated': None, 'url': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/metadata/preview.jpg', 'name': 'Static Preview - Sample Image', 'format': 'JPEG', 'package_id': 'df5e5f96-7347-4aae-8b97-8cf3cd352d68', 'created': '2022-09-01T08:09:23.243123', 'state': 'active', 'mimetype_inner

In [64]:
# Write the zipfile with the land cover data to a local folder

LC_zip_link = LC_gj['result']['resources'][2]['url']
#print(land_cover_zip)

LC_zip = requests.get(LC_zip_link)

with open(r'C:\Users\15612\Documents\GIS-5571\Lab2\land_cover.zip','wb') as LC_file:
    LC_file.write(LC_zip.content)
        
with zipfile.ZipFile(r"C:\Users\15612\Documents\GIS-5571\Lab2\land_cover.zip") as LC_zip:
    LC_zip.extractall(r'C:\Users\15612\Documents\GIS-5571\Lab2\land_cover')

In [56]:
# Add land cover data from local file path

m.addDataFromPath(r"C:\Users\15612\Documents\GIS-5571\Lab2\land_cover\NLCD_2019_Land_Cover.tif")

In [65]:
# Gather DEM data from Minnesota Geospatial Commons API

request = requests.get(
    'https://gisdata.mn.gov/api/3/action/package_show?id=elev-30m-digital-elevation-model',
    verify = False
)
    
DEM_gj = request.json()
print(DEM_gj)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'help': 'https://gisdata.mn.gov/api/3/action/help_show?name=package_show', 'success': True, 'result': {'license_title': 'License not specified', 'maintainer': None, 'relationships_as_object': [], 'private': False, 'maintainer_email': None, 'num_tags': 3, 'id': '8b8d727b-2d63-4f92-8aca-40752006f7af', 'metadata_created': '2014-07-21T14:39:36.374865', 'metadata_modified': '2022-09-01T08:08:42.496971', 'author': None, 'author_email': None, 'state': 'active', 'version': None, 'creator_user_id': None, 'type': 'dataset', 'resources': [{'mimetype': None, 'cache_url': None, 'hash': '', 'description': '', 'gdrsResGuid': '', 'cache_last_updated': None, 'url': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/preview/preview.jpg', 'name': 'Static Preview - Sample Image', 'format': 'JPEG', 'package_id': '8b8d727b-2d63-4f92-8aca-40752006f7af', 'created': '2022-09-01T08:08:42.793263', 'state': 'active', 'mimetype_inner': None, 'last_modified': None,

In [66]:
# Write the contents of the zipfile to a folder to acquire DEM

DEM_zip_link = DEM_gj['result']['resources'][1]['url']
print(DEM_zip_link)

DEM_zip = requests.get(DEM_zip_link)

with open(r'C:\Users\15612\Documents\GIS-5571\Lab2\DEM.zip','wb') as DEM_file:
    DEM_file.write(DEM_zip.content)
        
with zipfile.ZipFile(r"C:\Users\15612\Documents\GIS-5571\Lab2\DEM.zip") as DEM_zip:
    DEM_zip.extractall(r'C:\Users\15612\Documents\GIS-5571\Lab2\DEM')

https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip


In [68]:
# Add DEM data from local file

m.addDataFromPath(r"C:\Users\15612\Documents\GIS-5571\Lab2\DEM\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m")

In [19]:
# Create slope from DEM

DEM_Slope = arcpy.sa.SurfaceParameters(
    r'C:\Users\15612\Documents\GIS-5571\Lab2\DEM\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m',
    'SLOPE',
)

RuntimeError: ERROR 010240: Could not save raster dataset to C:\Users\15612\Documents\GIS-5571\Lab2\DEM\elev_30m_digital_elevation_model.gdb\DEM_Slope.tif with output format FGDBR.

In [7]:
# Reclassify values to different weights for different land cover types

for lyr in m.listLayers():
    if lyr.name == "NLCD_2019_Land_Cover.tif":
        farm_and_water_layer = arcpy.sa.Reclassify(
        lyr,
        'Value',
        arcpy.sa.RemapRange([[11,12,2],[21,24,1],[31,31,1],[41,43,1],[52,52,1],[71,71,1],[81,82,3],[90,90,2],[95,95,2]])
)
        
farm_and_water_layer.save(r'C:\Users\15612\Documents\GIS-5571\Lab2\farm_and_water_layer.tif')

In [23]:
# Reclassify slope to different weighted values based on severity of slope angle

for lyr in m.listLayers():
    if lyr.name == "DEM_Slope":
        steep_slope = arcpy.sa.Reclassify(
        lyr,
        'Value',
        arcpy.sa.RemapRange([[0,20,1],[20,50,2],[50,80.2473,3]])
)
        
steep_slope.save(r'C:\Users\15612\Documents\GIS-5571\Lab2\steep_slope.tif')

In [6]:
# Set variables equal to the steep_slope and farm_and_water_layer rasters

slope_layer = r'C:\Users\15612\Documents\GIS-5571\Lab2\steep_slope.tif'
farm_and_water = r'C:\Users\15612\Documents\GIS-5571\Lab2\farm_and_water_layer.tif'

In [25]:
# Create different rasters numbered 1-4 based on weights of 0.2, 0.4, 0.6, and 0.8

rast_num = 0
for num in range(1,5):
    rast_num += 1
    slope_import = round(num*0.2, 1)
    farm_import = round(1-num*0.2, 1)
    slope_rast = arcpy.sa.Raster(slope_layer) * slope_import
    farm_rast = arcpy.sa.Raster(farm_and_water) * farm_import
    slope_rast.save(r'C:\Users\15612\Documents\GIS-5571\Lab2\slope_rast_' + str(rast_num) + '.tif')
    farm_rast.save(r'C:\Users\15612\Documents\GIS-5571\Lab2\farm_rast_' + str(rast_num) + '.tif')

In [7]:
# Originally I was going to make the weights [0.2,0.8] and [0.4,0.6] for both rasters, but I feel I should add a raster where they both have equal weights

# To make the maximum value equivalent to the other rasters, multiply both by 0.5

slope_rast_equal = arcpy.sa.Raster(slope_layer) * 0.5
farm_rast_equal = arcpy.sa.Raster(farm_and_water) * 0.5
slope_rast_equal.save(r'C:\Users\15612\Documents\GIS-5571\Lab2\slope_rast_equal.tif')
farm_rast_equal.save(r'C:\Users\15612\Documents\GIS-5571\Lab2\farm_rast_equal.tif')

In [26]:
# Add the rasters to the map

for i in range(1,5):
    m.addDataFromPath(r'C:\Users\15612\Documents\GIS-5571\Lab2\slope_rast_' + str(i) + '.tif')
    m.addDataFromPath(r'C:\Users\15612\Documents\GIS-5571\Lab2\farm_rast_' + str(i) + '.tif')

In [3]:
# Create unequal cost surfaces by adding both rasters to each other

for i in range(1,5):
    cost_surface = arcpy.sa.RasterCalculator(
        [r'C:\Users\15612\Documents\GIS-5571\Lab2\slope_rast_' + str(i) + '.tif',r'C:\Users\15612\Documents\GIS-5571\Lab2\farm_rast_' + str(i) + '.tif'],
        ['x','y'],
        'x + y'
    )

    cost_surface.save(r'C:\Users\15612\Documents\GIS-5571\Lab2\cost_surface' + str(i) + '.tif')

In [8]:
# Create cost where slope and land cover are weighted equally

cost_surface_equal = arcpy.sa.RasterCalculator(
    [r'C:\Users\15612\Documents\GIS-5571\Lab2\slope_rast_equal.tif',r'C:\Users\15612\Documents\GIS-5571\Lab2\farm_rast_equal.tif'],
    ['x','y'],
    'x + y'
)

cost_surface_equal.save(r'C:\Users\15612\Documents\GIS-5571\Lab2\cost_surface_equal.tif')

In [4]:
# Add unequal cost surfaces to map

for i in range(1,5):
    m.addDataFromPath(r'C:\Users\15612\Documents\GIS-5571\Lab2\cost_surface' + str(i) + '.tif')

In [9]:
# Add equally weighted cost surface to map

m.addDataFromPath(r'C:\Users\15612\Documents\GIS-5571\Lab2\cost_surface_equal.tif')